# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,telciu,47.4333,24.4000,59.70,86,24,3.36,RO,1723084859
1,1,adamstown,-25.0660,-130.1015,67.05,65,94,3.91,PN,1723084355
2,2,alotau,-10.3069,150.4462,82.17,70,79,6.89,PG,1723084861
3,3,isafjordur,66.0755,-23.1240,44.15,87,100,2.10,IS,1723084758
4,4,guna,24.6500,77.3167,74.08,99,100,5.01,IN,1723084863


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat", 
    geo = True, 
    size = "Humidity", 
    tiles = "OSM")

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
# max temp lower than 27 degrees and humidity less than 72
ideal_df = city_data_df.loc[(city_data_df["Max Temp"] < 80)
                                       & (city_data_df["Humidity"] < 72)
                                      ]

# Drop any rows with null values
ideal_df.dropna()

# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,adamstown,-25.0660,-130.1015,67.05,65,94,3.91,PN,1723084355
5,5,blackmans bay,-43.0167,147.3167,55.72,65,100,3.00,AU,1723084353
9,9,bilibino,68.0546,166.4372,58.41,37,16,10.74,RU,1723084815
20,20,camapua,-19.5314,-54.0439,71.24,36,4,0.76,BR,1723084883
23,23,petropavlovsk-kamchatsky,53.0452,158.6483,79.39,47,75,13.42,RU,1723084887
...,...,...,...,...,...,...,...,...,...,...
550,550,staryye ozinki,51.1786,49.6769,70.50,65,26,4.76,RU,1723085525
560,560,susanville,40.4163,-120.6530,76.17,20,33,7.56,US,1723085539
563,563,worland,44.0169,-107.9554,69.67,61,75,21.85,US,1723085542
564,564,klyuchi,52.2667,79.1667,73.18,67,99,6.49,RU,1723085376


### Step 3: Create a new DataFrame called `hotel_df`.

In [15]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,adamstown,PN,-25.0660,-130.1015,65,
5,blackmans bay,AU,-43.0167,147.3167,65,
9,bilibino,RU,68.0546,166.4372,37,
20,camapua,BR,-19.5314,-54.0439,36,
23,petropavlovsk-kamchatsky,RU,53.0452,158.6483,47,
...,...,...,...,...,...,...
550,staryye ozinki,RU,51.1786,49.6769,65,
560,susanville,US,40.4163,-120.6530,20,
563,worland,US,44.0169,-107.9554,61,
564,klyuchi,RU,52.2667,79.1667,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
bilibino - nearest hotel: No hotel found
camapua - nearest hotel: No hotel found
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
edinburgh of the seven seas - nearest hotel: No hotel found
alofi - nearest hotel: Matavai Resort
dubbo - nearest hotel: Blue Gum Motel
udachny - nearest hotel: Вилюй
avarua - nearest hotel: Paradise Inn
port saint john's - nearest hotel: Outback Inn
vadso - nearest hotel: Scandic Vadsø
mata-utu - nearest hotel: Gîte Oceania
kegen - nearest hotel: Kegen
chernyshevskiy - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
vorgashor - nearest hotel: София
yellowknife - nearest hotel: No hotel found
tazovsky - nearest hotel: Тазовчанка
dudinka - nearest hotel: Виктория
margaret river - nearest hotel: Margaret River Hotel
mitsamiouli - nearest hotel: Hotel le Paradis
b

,City,Country,Lat,Lng,Humidity,Hotel Name
1,adamstown,PN,-25.0660,-130.1015,65,No hotel found
5,blackmans bay,AU,-43.0167,147.3167,65,Villa Howden
9,bilibino,RU,68.0546,166.4372,37,No hotel found
20,camapua,BR,-19.5314,-54.0439,36,No hotel found
23,petropavlovsk-kamchatsky,RU,53.0452,158.6483,47,ООО Постоялый двор
...,...,...,...,...,...,...
550,staryye ozinki,RU,51.1786,49.6769,65,No hotel found
560,susanville,US,40.4163,-120.6530,20,SureStay Plus Hotel by Best Western Susanville
563,worland,US,44.0169,-107.9554,61,Town House Motor Inn
564,klyuchi,RU,52.2667,79.1667,67,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)